<a href="https://colab.research.google.com/github/Carlos-Pessin/NeuralNetwork---Image-classification/blob/main/LAB_02C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import seaborn as sb
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Perceptron
from sklearn.metrics import accuracy_score, confusion_matrix
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt
from matplotlib import rcParams
import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense
import cv2
import skimage
import skimage.exposure
import skimage.transform
import pywt
import sys
import tensorflow as tf
sys.path.append('/content/drive/MyDrive/Bibs')
import bib_atributos
#import bibAtributos3D
K.set_image_data_format('channels_last')



In [ ]:
#%% funcoes
def gaussian(x, u, std):
	return K.exp(-1*(((0.5*(x-u))/std)**2))

In [ ]:
#%% Ler imagens para treinamento
numeroImagens = 20
I = np.zeros((315, 315, numeroImagens), dtype=np.float64)
GS = np.zeros((315, 315, numeroImagens), dtype=np.float64)

for i in range(numeroImagens):
	id = str(i)
	I[:,:,i] = skimage.img_as_float(cv2.imread('/content/drive/MyDrive/Pós/Redes Neurais/Lab 02C/I' + id + '.pgm',0))
	GS[:,:,i] = skimage.img_as_float(cv2.imread('/content/drive/MyDrive/Pós/Redes Neurais/Lab 02C/GSM' + id + '.pgm',0))

In [ ]:
#%% verifica se tamaho par

(M,N,ID) = np.shape(I)

if M % 2 == 1:
	M = M + 1
if N % 2 ==1:
	N = N + 1

size = [M,N]

Iresize = np.zeros((M,N,ID), dtype=np.float64)
GSresize = np.zeros((M,N,ID), dtype=np.float64)


for i in range(numeroImagens):
	img = I[:,:,i]
	gst = GS[:,:,i]

	img = skimage.transform.resize(img, output_shape = size)
	gst = skimage.transform.resize(gst, output_shape = size)

	Iresize[:,:,i] = img
	GSresize[:,:,i] = gst

I = Iresize
GS = GSresize

In [ ]:
#%% separa dados de treino

I_treino = I[:,:,1:20]
GS_treino = GS[:,:,1:20]

I_test = I[:,:,0:1]
GS_test = GS[:,:,0:1]

In [ ]:
#%% preprocessamento
treino_Atributo1= np.zeros((M,N,19), dtype=np.float64)

(cA, cDv, cDh, cDd) = bib_atributos.Coefs_Wavelet(I_treino)

for i in range(19):
  treino_Atributo1[:,:,i] = cA[:,:,i] + cDv[:,:,i] + cDh[:,:,i]

(cA, cDv, cDh, cDd) = bib_atributos.Coefs_Wavelet(I_test)
test_Atributo1 = cA + cDv + cDh


treino_Atributo2 = bib_atributos.Filt_High_Median(I_treino,5)
test_Atributo2 = bib_atributos.Filt_High_Median(I_test,5)

plt.figure()
plt.title('Imagem original')
plt.imshow(I_treino[:,:,0], cmap='gray')
plt.figure()
plt.title('Imagem com wavelet')
plt.imshow(treino_Atributo1[:,:,0], cmap='gray')
plt.figure()
plt.title('Imagem com Passa Alta Mediana')
plt.imshow(treino_Atributo2[:,:,0], cmap='gray')

plt.figure()
plt.title('Imagem original')
plt.imshow(I_test[:,:,0], cmap='gray')
plt.figure()
plt.title('Imagem com wavelet')
plt.imshow(test_Atributo1[:,:,0], cmap='gray')
plt.figure()
plt.title('Imagem com Passa Alta Mediana')
plt.imshow(test_Atributo2[:,:,0], cmap='gray')

In [ ]:
#%% Dados
# Transforma as imagens e colunas para a entrada da rede
Ivector_treino = I_treino.flatten(order='C')
Ivector_treino_Atributo1 = treino_Atributo1.flatten(order='C')
Ivector_treino_Atributo2 = treino_Atributo2.flatten(order='C')
GSvector_treino = GS_treino.flatten(order='C')

Ivector_test = I_test.flatten(order='C')
Ivector_test_Atributo1 = test_Atributo1.flatten(order='C')
Ivector_test_Atributo2 = test_Atributo2.flatten(order='C')
GSvector_test = GS_test.flatten(order='C')

X_treinamento = [Ivector_treino, Ivector_treino_Atributo1, Ivector_treino_Atributo2]
X_treinamento = np.transpose(X_treinamento)
y_treinamento = GSvector_treino

X_test = [Ivector_test, Ivector_test_Atributo1, Ivector_test_Atributo2]
X_test = np.transpose(X_test)
y_test = GSvector_test > 0.5

In [ ]:
#%% construcao do modelo
N_entradas = 3
inputs = tf.keras.layers.Input((N_entradas,))	# tensor de entrada

# camada de entrada 3 neuronio e com 3 entradas
# Layer 1
L1 = tf.keras.layers.Dense(30, kernel_initializer='ones', bias_initializer='zeros', input_shape=((N_entradas,)))(inputs)
L1 = gaussian(L1, u=1, std=0.05)

# Layer 2 - camada escondida
L2 = tf.keras.layers.Dense(30, kernel_initializer='ones', bias_initializer='zeros', input_shape=((N_entradas,)))(L1)
L2 = tf.keras.activations.relu(L2, alpha=0, max_value=1, threshold=0)

# camada de saida
Lout = tf.keras.layers.Dense(1, kernel_initializer='ones', bias_initializer='zeros', input_shape=((N_entradas,)))(L2)
Lout = tf.keras.activations.relu(Lout, alpha=0, max_value=1, threshold=0)

# compilar a rede
model = tf.keras.Model(inputs=[inputs], outputs = [Lout])
model.summary()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 3)]               0         
                                                                 
 dense (Dense)               (None, 30)                120       
                                                                 
 tf.math.subtract (TFOpLamb  (None, 30)                0         
 da)                                                             
                                                                 
 tf.math.multiply (TFOpLamb  (None, 30)                0         
 da)                                                             
                                                                 
 tf.math.truediv (TFOpLambd  (None, 30)                0         
 a)                                                              
                                                             

In [ ]:
#%% Treinamento
model.fit(X_treinamento, y_treinamento, validation_data=(X_test,y_test), batch_size=(N*M), epochs=1000)

In [ ]:
#%% Teste e avaliacao
y_pred = model.predict(X_test)
y_pred_bool = y_pred > 0.1

acc = accuracy_score(y_test, y_pred_bool)
matriz_conf = confusion_matrix(y_test, y_pred_bool)

print(acc)
print(matriz_conf)

3121/3121 [==============================] - 7s 2ms/step
0.997215991027079
[[99578     0]
 [  278     0]]


In [ ]:
#%% exibe imagens
Imatrix_treino = Ivector_treino.reshape(N, -1)
I_pred = y_pred.reshape(N, -1)
I_pred_bool = y_pred_bool.reshape(N, -1)

plt.figure()
plt.title('Gold standard')
plt.imshow(GS_test , cmap='gray')

plt.figure()
plt.title('Imagem predita')
plt.imshow(I_pred, cmap='gray')

plt.figure()
plt.title('Imagem predita Booleana')
plt.imshow(I_pred_bool, cmap='gray')